# ANEX A

A continuació, es mostren els diferents scripts que s'han utilitzat per a la creació del sistema de detecció de plagi. L'ordre dels scripts que es mostren és el següent:

1. Sistema de detecció de plagi mitjançant intel·ligència artificial
2. Models de classificació de documents en múltiples categories
3. Ajustament de les diferents combinacions d'encoder-mètrica de similitud per determinar l'índex de similitud semàntica
4. Distribució de freqüència de similitud entre oracions

Respecte l'script 3, només es mostra l'execució d'aquest en un dels 24 documents plagiats generats. En concret, en el Doc1.0, corresponent a la categoria mecànica de fluids. Aquest script pot ser extrapolat als 23 documents restants.

En el cas de l'script 4, només es mostra l'execució d'aquest per a la categoria electrònica. El codi és extrapolable a les demés categories.

# Sistema de detecció de plagi mitjançant intel·ligència artificial

### Importació de les biblioteques/dependències necessàries

In [1]:
import pandas as pd
import glob
import os
import pickle
import numpy as np 
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
import unidecode
import re
from sklearn import metrics
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity
import unidecode
from google.cloud import vision
import io
import shutil
from google.cloud import storage
import os
import json
import docx2txt

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/rogerlopezsantalo/Desktop/TFG/97_Keys/proyecto_personal/plagio-documentos-266509-4a5fc329ded0.json'

In [3]:
stemmer = SnowballStemmer('spanish')

### ESPECIFICAR PATH DEL DOCUMENT A ANALITZAR

In [16]:
path = '../06FUNCIONAMENT_SISTEMA/Arxiu_analitzar/Curvas características de una bomba hidráulica centrifuga.jpg'
path_doc_analitzar = path

### Càrrega de l'encoder i el model de classificació

In [17]:
#Encoder: CountVectorizer Binary
with open('../03Models/classificacio/cv_binary.pkl', 'rb') as file:
    pickle_feature = pickle.load(file)
    
#Model de classificació
with open('../03Models/classificacio/model_classificacio_mnb.pkl', 'rb') as file:
    pickle_model = pickle.load(file)

### Predicció de la categoria del document a analitzar

In [18]:
#Funció que permet executar la predicció a partir de la combinació encoder+model classificació (Cv binary + MLR)
def prediccion(titulo):
    titulo = titulo.split('/')[-1].split('.')[0]
    titulo = titulo.replace('_',' ')
    
    word_tokens = nltk.word_tokenize(titulo)
    text_without_stopwords= " ".join([w for w in word_tokens if not w in stopwords.words('spanish')])
    stemmed_text = " ".join(stemmer.stem(word) for word in nltk.word_tokenize(text_without_stopwords))
    no_punctuaction_text = re.sub(r'[^\w\s]','',stemmed_text)
    #no_punctuaction_text = unidecode.unidecode(stemmed_text)
    no_digits_text = re.sub('\d', '', no_punctuaction_text)
    X = pickle_feature.transform(pd.Series(no_digits_text))
    return(pickle_model.predict(X))

start = "\033[1m"
end = "\033[0;0m"

label = prediccion(path)
print('La categoria que prediu el model de classificació per al document amb títol: '+str(path.split('/')[-1].split('.')[0])+' es: '+start+label[0]+end)

La categoria que prediu el model de classificació per al document amb títol: Curvas características de una bomba hidráulica centrifuga es: Mecanica_fluids


### Càlcul de similitud semàntica textual del document a analitzar amb els demés documents de la base de dades que tenen assignada la mateixa categoria. 

In [15]:
#Funció que permet pre-processar el text dins de qualsevol tfg i separar-lo en sentences
def clean_raw_text (documento):
    text_sentences = []
    text = str(documento).replace('\n',' ')
    text = nltk.sent_tokenize(text)
    
    for sentence in text:
        words = []
        sentence = sentence.lower()
        word_tokens = nltk.word_tokenize(sentence)
        filtered_text = " ".join([w for w in word_tokens if not w in stopwords.words('spanish')])
        stemmed_text = " ".join(stemmer.stem(word) for word in nltk.word_tokenize(filtered_text))  
        no_punctuaction_text = re.sub(r'[^\w\s]','',stemmed_text)
        no_accents = unidecode.unidecode(no_punctuaction_text)
        no_digits_text = re.sub('\d', '', no_accents)
        for word in nltk.word_tokenize(no_digits_text):
            if len(word) >=2:
                words.append(word)
                                
        no_digits_text = " ".join(words)
        clean1 = re.sub(' +', ' ',no_digits_text)
        full_clean = clean1.strip()    
        text_sentences.append(full_clean)
        
    text_sentences = [x for x in text_sentences if x !='']
    return(text_sentences)

#Funció que permet descarregar un objecte de Google Storage
def download_blob(bucket_name, source_blob_name, destination_file_name):
        """Downloads a blob from the bucket."""
        # bucket_name = "your-bucket-name"
        # source_blob_name = "storage-object-name"
        # destination_file_name = "local/path/to/file"

        storage_client = storage.Client()

        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(source_blob_name)
        blob.download_to_filename(destination_file_name)

        print(
            "Descarregant arxiu: {} localment. \n".format(
                source_blob_name, destination_file_name
            )
        )

#Funció que permet carregar un objecte a Google Storage        
def upload_blob(bucket_name, source_file_name, destination_blob_name):
        """Uploads a file to the bucket."""
        # bucket_name = "your-bucket-name"
        # source_file_name = "local/path/to/file"
        # destination_blob_name = "storage-object-name"

        storage_client = storage.Client(project='plagio-documentos-266509')
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(destination_blob_name)

        blob.upload_from_filename(source_file_name)

        print(
            "Arxiu {} pujat a storage amb el nom: {}. \n".format(
                source_file_name, destination_blob_name
            )
        )

#Funció que permet eliminar un objecte de Google Storage
def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    # bucket_name = "your-bucket-name"
    # blob_name = "your-object-name"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.delete()

    print("Arxiu {} eliminat.\n".format(blob_name))

#FUNCIÓ COMPARACIÓ DOCUMENT A ANALITZAR VS DOCUMENTS DE LA MATEIXA CATEGORIA ASSIGNADA
def comparacio_doc_analitzar_vs_base_dades(path_doc_analitzar,categoria):
    
    
    #PELS DOCUMENTS A ANALITZAR .PDF
    if path_doc_analitzar.split('/')[-1].split('.')[-1] == 'pdf':

        client = storage.Client(project='plagio-documentos-266509')
        upload_blob('arxius_analitzar','../06FUNCIONAMENT_SISTEMA/Arxiu_analitzar/'+path_doc_analitzar.split('/')[-1],path_doc_analitzar.split('/')[-1])
        doc_extreure = []
        for blob in client.list_blobs('arxius_analitzar'): #!!! Canviar el prefix per 'dataset_pdf' ja que 'original_dataset' es el de proves
            doc_extreure.append(blob.name)
        
        
        #Crea un o varis json a la carpeta 'dataset_json/[Sub_carpeta_especifica]' dins del bucket 'dataset_tfg' per a cada document. 
        #El json conté tot el text del document

        for file in doc_extreure:
            gcs_source_uri = 'gs://arxius_analitzar/'+file
            gcs_destination_uri = 'gs://json_analitzar/'+file

            """OCR with PDF/TIFF as source files on GCS"""
            # Supported mime_types are: 'application/pdf' and 'image/tiff' in case of DOCUMENT_TEXT_DETECTION
         
            mime_type = 'application/pdf'

            # How many pages should be grouped into each json output file.
            batch_size = 100 ## Número de pàgines que seran agrupades en un mateix json file. El màxim és 100
            client = vision.ImageAnnotatorClient()

            feature = vision.types.Feature(type=vision.enums.Feature.Type.DOCUMENT_TEXT_DETECTION)

            gcs_source = vision.types.GcsSource(uri=gcs_source_uri)
            input_config = vision.types.InputConfig(gcs_source=gcs_source, mime_type=mime_type)

            gcs_destination = vision.types.GcsDestination(uri=gcs_destination_uri)
            output_config = vision.types.OutputConfig(gcs_destination=gcs_destination, batch_size=batch_size)

            async_request = vision.types.AsyncAnnotateFileRequest(features=[feature], input_config=input_config,output_config=output_config)

            operation = client.async_batch_annotate_files(requests=[async_request])

            print('Extraient text del document mitjantçant Google API VISION. \n')
            operation.result(timeout=600)
            
            
        #Descarrego arxiu Json localment
        nom_json_descarregar = []
        storage_client = storage.Client()
        for blob in storage_client.list_blobs('json_analitzar'):
            nom_json_descarregar.append(blob.name)
            
        download_blob('json_analitzar',nom_json_descarregar[0],'../06FUNCIONAMENT_SISTEMA/json_analitzar/'+nom_json_descarregar[0].split('.pdf')[0]+'.json')
        
        #Elimino arxiu .pdf i .json del cloud  
        nom_pdf_eliminar = []
        storage_client = storage.Client()
        for blob in storage_client.list_blobs('arxius_analitzar'):
            nom_pdf_eliminar.append(blob.name)
            
        delete_blob('arxius_analitzar',nom_pdf_eliminar[0])
        delete_blob('json_analitzar',nom_json_descarregar[0])
            
        #Extrec el text de l'arxiu json a un arxiu txt generat localment 
        json_doc_list = (glob.glob("../06FUNCIONAMENT_SISTEMA/json_analitzar/*"))  
        
        for file in json_doc_list:
            texto = file.split('/')[-1]

            with open(file,'r') as json_file:
                values = json.load(json_file)


            with open('../06FUNCIONAMENT_SISTEMA/txt_analitzar/'+file.split('/')[-1].split('.json')[0]+'.txt','w+') as f:
                try:   
                    for i in range(0,len(values['responses'])):
                        try:
                            f.write(values['responses'][i]['fullTextAnnotation']['text'])
                        except KeyError as e:
                            if(e.args[0] != 'fullTextAnnotation'):
                                print(e)
                                raise(e)   
                except:
                    print(file)
        print('Text extret a un arxiu .txt \n')
        
        
    #PELS DOCUMENTS .DOCX
    elif path_doc_analitzar.split('/')[-1].split('.')[-1] == 'docx':
        result = docx2txt.process(path_doc_analitzar)
        result = result.replace('\n',' ')
        result = " ".join(result.split())
        print('Extraient el text del document a un arxiu .txt \n')
        with open('../06FUNCIONAMENT_SISTEMA/txt_analitzar/'+path_doc_analitzar.split('/')[-1].split('.')[0]+'.txt','w+') as file:
            file.write(result)
        print('Text extret a un arxiu .txt \n')
            
            
    #PELS DOCUMENTS .JPEG, .PNG8,.PNG24,.GIF
    else:
        print('Extraient el text a un arxiu .txt \n')
        client = vision.ImageAnnotatorClient()
        with io.open(path_doc_analitzar, 'rb') as image_file:
            content = image_file.read()
        image = vision.types.Image(content=content)
        response = client.text_detection(image=image)
        
        
        with open('../06FUNCIONAMENT_SISTEMA/txt_analitzar/'+path_doc_analitzar.split('/')[-1].split('.')[0]+'.txt','w+') as file:
            file.write(response.text_annotations[0].description.replace('\n',' '))
        print('Text extret a un arxiu .txt \n')
        
        
        
        
   
    #CÀLCUL DE L'ÍNDEX DE SIMILITUD SEMÀNTICA 
    print('Inici del càlcul de la similitud semàntica entre el document a analitzar i els documents de la categoria: '+start+label[0]+end)    
    
    path_txt_doc_analitzar = '../06FUNCIONAMENT_SISTEMA/txt_analitzar/*'
    #Carrego el text del document a analitzar
    llista_text_documents_plagiats = []
    ###!!!1 MIRAR SI FUNCIONA BÉ EL PATH JA QUE ESTIC AGAFANT EL TXT GENERAT DE LA EXTRACCIÓ
    for file in glob.glob(path_txt_doc_analitzar):
        with open(file,'r') as txt:
            llista_text_documents_plagiats.append(txt.read())
    
    #Extrec les oracions del document a analitzar
    plag_document_sentences = []
    for i in range (0,len(llista_text_documents_plagiats)):
        plag_document_sentences.append(clean_raw_text(llista_text_documents_plagiats[i]))
    
   
    #############################
    if categoria =='Automobil':
        
        #LListo tots els tfg dins de la categoria automobil
        llista_text_documents_automobil = []
        for file in glob.glob('../00Data/dataset_txt/Automobil/*'):
            with open(file,'r') as txt:
                llista_text_documents_automobil.append(txt.read())
                
        #Pre-processo i separo en oracions els documents de la categoria automobil
        base_document_sentences = []
        for i in range (0,len(llista_text_documents_automobil)):
            base_document_sentences.append(clean_raw_text(llista_text_documents_automobil[i]))

        #Carrego el model word2vec entrenat amb el vocabulari de la categoria automobil  
        with open('../03Models/w2v_models_for_compute_similarities/w2v_semantic_text_similarity_automobil.pkl', 'rb') as file:
            w2v_model = pickle.load(file)
        
        #Creo una llista de llistes on tinc les diferents frases tokenitzades de cada document de la base de dades. [[[]]]
        base_documents_frases_tokenitzades = []
        for i in range (0,len(base_document_sentences)):
            frases_tokenitzades = []
            for sentence in base_document_sentences[i]:
                frases_tokenitzades.append(nltk.word_tokenize(sentence))#Tokenitzo cada frase, obtenint una llista de paraules per a cada frase
            base_documents_frases_tokenitzades.append(frases_tokenitzades)

        #Creo una llista de llistes on tinc les diferents frases tokenitzades del document a analitzar.
        plag_document_frases_tokenitzades = []
        for sentence in plag_document_sentences[0]: 
            plag_document_frases_tokenitzades.append(nltk.word_tokenize(sentence))

        #Filtro les paraules dels documents base de cada frase segons si pertanyen al vocabulari del word2vec
        base_filtered_document_sentence_global = []
        for document in base_documents_frases_tokenitzades: 
            filtered_sentence_global=[] 
            for sentence in document: 
                filtered_sentence_global.append([word for word in sentence if word in w2v_model.wv.vocab])
            base_filtered_document_sentence_global.append(filtered_sentence_global)

        #Filtro les paraules de les frases del document a analitzar segons si pertanyen al vocabulari del word2vec
        plag_filtered_document_sentence = []
        for sentence in plag_document_frases_tokenitzades:
            plag_filtered_document_sentence.append([word for word in sentence if word in w2v_model.wv.vocab])
            
        #Transformo a vectors les paraules de cada frase de cada document de la base de dades a partir del wor2vec.
        #No ha estat necessari filtrar les paraules perquè totes ja formen part del vocabulari. Si no fos així, hauria de filtrar-les 
        base_document_sentence_vectors = []
        for i in range (0,len(base_filtered_document_sentence_global)):
            frase_vectorizada = []
            for sentence in base_filtered_document_sentence_global[i]:
                mean = []
                for word in sentence:
                    mean.append(w2v_model.wv.get_vector(word))
                mean = np.array(mean).mean(axis=0)
                frase_vectorizada.append(mean)
            base_document_sentence_vectors.append(frase_vectorizada)

        #Transformo a vectors les paraules de cada frase del document a analitzar a partir del wor2vec.
        plag_document_sentence_vectors = []
        for sentence in plag_filtered_document_sentence:
            mean = []
            for word in sentence:
                mean.append(w2v_model.wv.get_vector(word))
            mean = np.array(mean).mean(axis=0)
            plag_document_sentence_vectors.append(mean)
            
        #Transformo els vectors dels documents de la base de dades d'una dimensió a dues dimensions per tal de poder aplicar posteriorment cosine similarity
        base_document_sentence_vectors_good_format = []
        for document in base_document_sentence_vectors:
            vectorizado_bueno = []
            for vector in document:
                vectorizado_bueno.append([vector])
            base_document_sentence_vectors_good_format.append(vectorizado_bueno)
            
        #Transformo els vectors del document a analitzar d'una dimensió a dues dimensions per tal de poder aplicar posteriorment cosine similarity
        plag_document_sentence_vectors_good_format = []
        for vector in plag_document_sentence_vectors:
            plag_document_sentence_vectors_good_format.append([vector])
    
        #Computo les similituds entre els vectors del document a analitzar i els vectors de cada document de la base de dades mitjançant cosine similarity
        similarities_between_each_vector_plag_each_vector_base = []
        for i in range (0,len(base_document_sentence_vectors_good_format)):
            each_document = []
            for vector2 in plag_document_sentence_vectors_good_format:
                each_plag_sentence = []
                for vector1 in base_document_sentence_vectors_good_format[i]:
                    each_plag_sentence.append(cosine_similarity(vector2,vector1))
                each_document.append(each_plag_sentence)
            similarities_between_each_vector_plag_each_vector_base.append(each_document)    
    
        #Extrec les màximes similituds dels documents de la base de dades per a cada frase del document a analitzar
        best_similarities_sentence_document = []
        for i in range (0,len(similarities_between_each_vector_plag_each_vector_base)):
            each_document = []
            for each_vector in similarities_between_each_vector_plag_each_vector_base[i]:
                each_document.append(max(each_vector))
            best_similarities_sentence_document.append(each_document)

        #Calculo l'index de similitud entre el document a analitzar i els documents de la base de dades
        index_entre_documents_w2v = []
        for each_document in best_similarities_sentence_document:
            index_entre_documents_w2v.append(sum(each_document)/len(each_document))
                
        #Llindar de plagi fixat a partir de la distribució de freqüències de similitud entre oracions dels documents de la base de dades corresponents a la categoria automòbil
        if max(index_entre_documents_w2v) > 0.35:
            return('El document de la base de dades titulat: '+str(glob.glob('../00Data/dataset_txt/Automobil/*')[index_entre_documents_w2v.index(max(index_entre_documents_w2v))].split('/')[-1])+' te un percentatge de similitud del: '+str(max(index_entre_documents_w2v)[0])+' respecte al document a analitzar')
    
        else:
            return('No s''ha detectat cap document a la base de dades que tingui una semblança major o igual al 35% i que pugui ser considerat objecte de plagi.')
    #############################
    if categoria =='Mecanica_fluids':
        
        #LListo tots els tfg dins de la categoria automobil
        llista_text_documents_automobil = []
        for file in glob.glob('../00Data/dataset_txt/Mecanica_fluids/*'):
            with open(file,'r') as txt:
                llista_text_documents_automobil.append(txt.read())
                        
        #Pre-processo i separo en oracions els documents de la categoria automobil
        base_document_sentences = []
        for i in range (0,len(llista_text_documents_automobil)):
            base_document_sentences.append(clean_raw_text(llista_text_documents_automobil[i]))
            
        #Carrego el model word2vec entrenat amb el vocabulari de la categoria automobil  
        with open('../03Models/w2v_models_for_compute_similarities/w2v_semantic_text_similarity_mecanica_fluids.pkl', 'rb') as file:
            w2v_model = pickle.load(file)
        
        #Creo una llista de llistes on tinc les diferents frases tokenitzades de cada document de la base de dades. [[[]]]
        base_documents_frases_tokenitzades = []
        for i in range (0,len(base_document_sentences)):
            frases_tokenitzades = []
            for sentence in base_document_sentences[i]:
                frases_tokenitzades.append(nltk.word_tokenize(sentence))#Tokenitzo cada frase, obtenint una llista de paraules per a cada frase
            base_documents_frases_tokenitzades.append(frases_tokenitzades)

        #Creo una llista de llistes on tinc les diferents frases tokenitzades del document a analitzar
        plag_document_frases_tokenitzades = []
        for sentence in plag_document_sentences[0]: 
            plag_document_frases_tokenitzades.append(nltk.word_tokenize(sentence))
     
        #Filtro les paraules dels documents base de cada frase segons si pertanyen al vocabulari del word2vec
        base_filtered_document_sentence_global = []
        for document in base_documents_frases_tokenitzades: 
            filtered_sentence_global=[] 
            for sentence in document: 
                filtered_sentence_global.append([word for word in sentence if word in w2v_model.wv.vocab])
            base_filtered_document_sentence_global.append(filtered_sentence_global)

        #Filtro les paraules del document a analitzar segons si pertanyen al vocabulari del word2vec
        plag_filtered_document_sentence = []
        for sentence in plag_document_frases_tokenitzades:
            plag_filtered_document_sentence.append([word for word in sentence if word in w2v_model.wv.vocab])
 
        #Transformo a vectors les paraules de cada frase de cada document de la base de dades a partir del wor2vec i genero un vector per a cada frase
        #No ha estat necessari filtrar les paraules perquè totes ja formen part del vocabulari. Si no fos així, hauria de filtrar-les
        base_document_sentence_vectors = []
        for i in range (0,len(base_filtered_document_sentence_global)):
            frase_vectorizada = []
            for sentence in base_filtered_document_sentence_global[i]:
                mean = []
                for word in sentence:
                    mean.append(w2v_model.wv.get_vector(word))
                mean = np.array(mean).mean(axis=0)
                frase_vectorizada.append(mean)
            base_document_sentence_vectors.append(frase_vectorizada)

        #Transformo a vectors les paraules de cada frase del document a analitzar a partir del word2vec i genero un vector per a cada frase
        plag_document_sentence_vectors = []
        for sentence in plag_filtered_document_sentence:
            mean = []
            for word in sentence:
                mean.append(w2v_model.wv.get_vector(word))
            mean = np.array(mean).mean(axis=0)
            plag_document_sentence_vectors.append(mean)

        #Transformo els vectors d'una dimensió a dues dimensions per tal de poder aplicar posteriorment cosine similarity
        base_document_sentence_vectors_good_format = []
        for document in base_document_sentence_vectors:
            vectorizado_bueno = []
            for vector in document:
                vectorizado_bueno.append([vector])
            base_document_sentence_vectors_good_format.append(vectorizado_bueno)

        #Transformo els vectors del document a analitzar d'una dimensió a dues dimensions per tal de poder aplicar posteriorment cosine similarity
        plag_document_sentence_vectors_good_format = []
        for vector in plag_document_sentence_vectors:
            plag_document_sentence_vectors_good_format.append([vector])
            
        #Computo les similituds entre els vectors del document a analitzar i els vectors de cada document de la base de dades mitjançant cosine similarity
        similarities_between_each_vector_plag_each_vector_base = []
        for i in range (0,len(base_document_sentence_vectors_good_format)):
            each_document = []
            for vector2 in plag_document_sentence_vectors_good_format:
                each_plag_sentence = []
                for vector1 in base_document_sentence_vectors_good_format[i]:
                    each_plag_sentence.append(cosine_similarity(vector2,vector1))
                each_document.append(each_plag_sentence)
            similarities_between_each_vector_plag_each_vector_base.append(each_document)
    
        #Extrec les màximes similituds dels documents de la base de dades per a cada frase del document a analitzar
        best_similarities_sentence_document = []
        for i in range (0,len(similarities_between_each_vector_plag_each_vector_base)):
            each_document = []
            for each_vector in similarities_between_each_vector_plag_each_vector_base[i]:
                each_document.append(max(each_vector))
            best_similarities_sentence_document.append(each_document)

        #Calculo l'index de similitud entre el document a analitzar i els documents de la base de dades de la categoria mecanica de fluids
        index_entre_documents_w2v = []
        for each_document in best_similarities_sentence_document:
            index_entre_documents_w2v.append(sum(each_document)/len(each_document))
              
        if max(index_entre_documents_w2v) > 0.36: 
            return('El document de la base de dades titulat: '+str(glob.glob('../00Data/dataset_txt/Mecanica_fluids/*')[index_entre_documents_w2v.index(max(index_entre_documents_w2v))].split('/')[-1])+' te un percentatge de similitud del: '+str(max(index_entre_documents_w2v)[0])+' respecte al document a analitzar')
        else:
            return('No s''ha detectat cap document a la base de dades que tingui una semblança major o igual al 36% i que pugui ser considerat objecte de plagi.')  
    #############################
    if categoria =='Electronica':
        
        #LListo tots els tfg dins de la categoria automobil
        llista_text_documents_automobil = []
        for file in glob.glob('../00Data/dataset_txt/Electronica/*'):
            with open(file,'r') as txt:
                llista_text_documents_automobil.append(txt.read())
                         
        #Pre-processo i separo en oracions els documents de la categoria automobil
        base_document_sentences = []
        for i in range (0,len(llista_text_documents_automobil)):
            base_document_sentences.append(clean_raw_text(llista_text_documents_automobil[i]))

        #Carrego el model word2vec entrenat amb el vocabulari de la categoria automobil  
        with open('../03Models/w2v_models_for_compute_similarities/w2v_semantic_text_similarity_electronica.pkl', 'rb') as file:
            w2v_model = pickle.load(file)
        
        #Creo una llista de llistes de llistes on tinc les diferents frases tokenitzades de cada document de la base de dades. [[[]]]
        base_documents_frases_tokenitzades = []
        for i in range (0,len(base_document_sentences)):
            frases_tokenitzades = []
            for sentence in base_document_sentences[i]:
                frases_tokenitzades.append(nltk.word_tokenize(sentence))#Tokenitzo cada frase, obtenint una llista de paraules per a cada frase
            base_documents_frases_tokenitzades.append(frases_tokenitzades)

        #Creo una llista de llistes on tinc les diferents frases tokenitzades del document a analitzar.
        plag_document_frases_tokenitzades = []
        for sentence in plag_document_sentences[0]: 
            plag_document_frases_tokenitzades.append(nltk.word_tokenize(sentence))

        #Filtro les paraules dels documents base de cada frase segons si pertanyen al vocabulari del word2vec
        base_filtered_document_sentence_global = []
        for document in base_documents_frases_tokenitzades: 
            filtered_sentence_global=[] 
            for sentence in document: 
                filtered_sentence_global.append([word for word in sentence if word in w2v_model.wv.vocab])
            base_filtered_document_sentence_global.append(filtered_sentence_global)

        #Filtro les paraules del document a analitzar segons si pertanyen al vocabulari del word2vec
        plag_filtered_document_sentence = []
        for sentence in plag_document_frases_tokenitzades:
            plag_filtered_document_sentence.append([word for word in sentence if word in w2v_model.wv.vocab])

        #Transformo a vectors les paraules de cada frase de cada document a partir del wor2vec i genereo un vector per a cada frase.
        #No ha estat necessari filtrar les paraules perquè totes ja formen part del vocabulari. Si no fos així, hauria de filtrar-les
        base_document_sentence_vectors = []
        for i in range (0,len(base_filtered_document_sentence_global)):
            frase_vectorizada = []
            for sentence in base_filtered_document_sentence_global[i]:
                mean = []
                for word in sentence:
                    mean.append(w2v_model.wv.get_vector(word))
                mean = np.array(mean).mean(axis=0)
                frase_vectorizada.append(mean)
            base_document_sentence_vectors.append(frase_vectorizada)


        #Transformo a vectors les paraules de cada frase del document a partir del word2vec i genero un vector per a cada frase.
        plag_document_sentence_vectors = []
        for sentence in plag_filtered_document_sentence:
            mean = []
            for word in sentence:
                mean.append(w2v_model.wv.get_vector(word))
            mean = np.array(mean).mean(axis=0)
            plag_document_sentence_vectors.append(mean)

        #Transformo els vectors dels documents base d'una dimensió a dues dimensions per tal de poder aplicar posteriorment cosine similarity
        base_document_sentence_vectors_good_format = []
        for document in base_document_sentence_vectors:
            vectorizado_bueno = []
            for vector in document:
                vectorizado_bueno.append([vector])
            base_document_sentence_vectors_good_format.append(vectorizado_bueno)

        #Transformo els vectors del document a analitzar d'una dimensió a dues dimensions per tal de poder aplicar posteriorment cosine similarity
        plag_document_sentence_vectors_good_format = []
        for vector in plag_document_sentence_vectors:
            plag_document_sentence_vectors_good_format.append([vector])
    
        #Computo les similituds entre els vectors del document a analitzar i els vectors de cada document de la base de dades mitjançant cosine similarity
        similarities_between_each_vector_plag_each_vector_base = []
        for i in range (0,len(base_document_sentence_vectors_good_format)):
            each_document = []
            for vector2 in plag_document_sentence_vectors_good_format:
                each_plag_sentence = []
                for vector1 in base_document_sentence_vectors_good_format[i]:
                    each_plag_sentence.append(cosine_similarity(vector2,vector1))
                each_document.append(each_plag_sentence)
            similarities_between_each_vector_plag_each_vector_base.append(each_document)
    
        #Extrec les màximes similituds dels documents de la base de dades per a cada frase del document a analitzar
        best_similarities_sentence_document = []
        for i in range (0,len(similarities_between_each_vector_plag_each_vector_base)):
            each_document = []
            for each_vector in similarities_between_each_vector_plag_each_vector_base[i]:
                each_document.append(max(each_vector))
            best_similarities_sentence_document.append(each_document)
                  
        #Calculo l'index de similitud entre el document a analitzar i els documents de la base de dades de la categoria electronica
        index_entre_documents_w2v = []
        for each_document in best_similarities_sentence_document:
            index_entre_documents_w2v.append(sum(each_document)/len(each_document))
        
        if max(index_entre_documents_w2v)>0.44:   
            return('El document de la base de dades titulat: '+str(glob.glob('../00Data/dataset_txt/Electronica/*')[index_entre_documents_w2v.index(max(index_entre_documents_w2v))].split('/')[-1])+' te un percentatge de similitud del: '+str(max(index_entre_documents_w2v)[0])+' respecte al document a analitzar')
    
        else:
            return('No s''ha detectat cap document a la base de dades que tingui una semblança major o igual al 44% i que pugui ser considerat objecte de plagi.')                      
    #############################
                                             
    if categoria =='Biomedicina':
        
        #LListo tots els tfg dins de la categoria biomedicina
        llista_text_documents_automobil = []
        for file in glob.glob('../00Data/dataset_txt/Biomedicina/*'):
            with open(file,'r') as txt:
                llista_text_documents_automobil.append(txt.read())
         #Pre-processo i separo en oracions els documents de la categoria biomedicina
        base_document_sentences = []
        for i in range (0,len(llista_text_documents_automobil)):
            base_document_sentences.append(clean_raw_text(llista_text_documents_automobil[i]))

        #Carrego el model word2vec entrenat amb el vocabulari de la categoria biomedicina
        with open('../03Models/w2v_models_for_compute_similarities/w2v_semantic_text_similarity_biomedicina.pkl', 'rb') as file:
            w2v_model = pickle.load(file)
        
        #Creo una llista de llistes on tinc les diferents frases tokenitzades de cada document. [[[]]]
        base_documents_frases_tokenitzades = []
        for i in range (0,len(base_document_sentences)):
            frases_tokenitzades = []
            for sentence in base_document_sentences[i]:
                frases_tokenitzades.append(nltk.word_tokenize(sentence))#Tokenitzo cada frase, obtenint una llista de paraules per a cada frase
            base_documents_frases_tokenitzades.append(frases_tokenitzades)

        #Creo una llista de llistes on tinc les diferents frases tokenitzades del document a analitzar
        plag_document_frases_tokenitzades = []
        for sentence in plag_document_sentences[0]: 
            plag_document_frases_tokenitzades.append(nltk.word_tokenize(sentence))

        #Filtro les paraules dels documents base de cada frase segons si pertanyen al vocabulari del word2vec
        base_filtered_document_sentence_global = []
        for document in base_documents_frases_tokenitzades: 
            filtered_sentence_global=[] 
            for sentence in document: 
                filtered_sentence_global.append([word for word in sentence if word in w2v_model.wv.vocab])
            base_filtered_document_sentence_global.append(filtered_sentence_global)

        #Filtro les paraules del document plagiat segons si pertanyen al vocabulari del word2vec
        plag_filtered_document_sentence = []
        for sentence in plag_document_frases_tokenitzades:
            plag_filtered_document_sentence.append([word for word in sentence if word in w2v_model.wv.vocab])

        #Transformo a vectors les paraules de cada frase de cada document a partir del wor2vec.
        #No ha estat necessari filtrar les paraules perquè totes ja formen part del vocabulari. Si no fos així, hauria de filtrar-les
        base_document_sentence_vectors = []
        for i in range (0,len(base_filtered_document_sentence_global)):
            frase_vectorizada = []
            for sentence in base_filtered_document_sentence_global[i]:
                mean = []
                for word in sentence:
                    mean.append(w2v_model.wv.get_vector(word))
                mean = np.array(mean).mean(axis=0)
                frase_vectorizada.append(mean)
            base_document_sentence_vectors.append(frase_vectorizada)

        #Transformo les frases del document a analitzar a vectors a partir de la mitjana dels vectors de les paraules que conformen les frases
        plag_document_sentence_vectors = []
        for sentence in plag_filtered_document_sentence:
            mean = []
            for word in sentence:
                mean.append(w2v_model.wv.get_vector(word))
            mean = np.array(mean).mean(axis=0)
            plag_document_sentence_vectors.append(mean)

        #Transformo els vectors d'una dimensió a dues dimensions per tal de poder aplicar posteriorment cosine similarity
        base_document_sentence_vectors_good_format = []
        for document in base_document_sentence_vectors:
            vectorizado_bueno = []
            for vector in document:
                vectorizado_bueno.append([vector])
            base_document_sentence_vectors_good_format.append(vectorizado_bueno)

        #Transformo els vectors d'una dimensió a dues dimensions per tal de poder aplicar posteriorment cosine similarity
        plag_document_sentence_vectors_good_format = []
        for vector in plag_document_sentence_vectors:
            plag_document_sentence_vectors_good_format.append([vector])
    
        #Computo les similituds entre els vectors del document a analitzar i els vectors de cada document de la base de dades mitjançant cosine similarity
        similarities_between_each_vector_plag_each_vector_base = []
        for i in range (0,len(base_document_sentence_vectors_good_format)):
            each_document = []
            for vector2 in plag_document_sentence_vectors_good_format:
                each_plag_sentence = []
                for vector1 in base_document_sentence_vectors_good_format[i]:
                    each_plag_sentence.append(cosine_similarity(vector2,vector1))
                each_document.append(each_plag_sentence)
            similarities_between_each_vector_plag_each_vector_base.append(each_document)
    
        #Extrec les màximes similituds dels documents de la base de dades per a cada frase del document a analitzar
        best_similarities_sentence_document = []
        for i in range (0,len(similarities_between_each_vector_plag_each_vector_base)):
            each_document = []
            for each_vector in similarities_between_each_vector_plag_each_vector_base[i]:
                each_document.append(max(each_vector))
            best_similarities_sentence_document.append(each_document)

        #Calculo l'index de similitud entre el document a analitzar i els documents de la base de dades
        index_entre_documents_w2v = []
        for each_document in best_similarities_sentence_document:
            index_entre_documents_w2v.append(sum(each_document)/len(each_document))
            
        
    
        
        if max(index_entre_documents_w2v) > 0.38: #Canviar % del llindar que estableixi la dist freq de biomedicina
            return('El document de la base de dades titulat: '+str(glob.glob('../00Data/dataset_txt/Biomedicina/*')[index_entre_documents_w2v.index(max(index_entre_documents_w2v))].split('/')[-1])+' te un percentatge de similitud del: '+str(max(index_entre_documents_w2v)[0])+' respecte al document a analitzar')
    
        else:
            return('No s''ha detectat cap document a la base de dades que tingui una semblança major o igual al 38% i que pugui ser considerat objecte de plagi.')
    

In [16]:
%%time
comparacio_doc_analitzar_vs_base_dades('../06FUNCIONAMENT_SISTEMA/Arxiu_analitzar/'+path.split('/')[-1],label[0])

Arxiu ../06FUNCIONAMENT_SISTEMA/Arxiu_analitzar/La_implementacion_de_los_robots_electronicos_en_nuestras_casas.pdf pujat a storage amb el nom: La_implementacion_de_los_robots_electronicos_en_nuestras_casas.pdf. 

Extraient text del document mitjantçant Google API VISION. 

Descarregant arxiu: La_implementacion_de_los_robots_electronicos_en_nuestras_casas.pdfoutput-1-to-1.json localment. 

Arxiu La_implementacion_de_los_robots_electronicos_en_nuestras_casas.pdf eliminat.

Arxiu La_implementacion_de_los_robots_electronicos_en_nuestras_casas.pdfoutput-1-to-1.json eliminat.

Text extret a un arxiu .txt 

Inici del càlcul de la similitud semàntica entre el document a analitzar i els documents de la categoria: Electronica
CPU times: user 1min 55s, sys: 12.6 s, total: 2min 7s
Wall time: 2min 30s


'El document de la base de dades titulat: Estacion_robotizada_de_paletizado te un percentatge de similitud del: [0.5358954] respecte al document a analitzar'

### Neteja de les carpetes locals dels arxius a analitzar

In [21]:
shutil.move('../06FUNCIONAMENT_SISTEMA/Arxiu_analitzar/'+path.split('/')[-1], '../06FUNCIONAMENT_SISTEMA/Arxius_analitzats/')
if len(glob.glob('../06FUNCIONAMENT_SISTEMA/json_analitzar/*'))>=1:
    shutil.move('../06FUNCIONAMENT_SISTEMA/json_analitzar/'+path.split('/')[-1].split('.')[0]+'.json', '../06FUNCIONAMENT_SISTEMA/jsons_analitzats/')
shutil.move('../06FUNCIONAMENT_SISTEMA/txt_analitzar/'+path.split('/')[-1].split('.')[0]+'.txt', '../06FUNCIONAMENT_SISTEMA/txts_analitzats/')
print('Els arxius han estat moguts a les carpetes "ANALITZATS"')

Els arxius han estat moguts a les carpetes "ANALITZATS"
